In [1]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 4.9 MB/s eta 0:00:00


For Single PDF

In [134]:
# @title
from pypdf import PdfReader
import re
import pandas as pd

# Constants for file paths
INPUT_PATH = "acchq.pdf"
OUTPUT_PATH = "extracted_text.txt"
CSV_OUTPUT_PATH = "extracted_data.csv"

# Read PDF
reader = PdfReader(INPUT_PATH)

# Function to apply patterns
def apply_patterns(text, patterns):
    """Apply regex patterns and print the number of changes for each pattern."""
    total_changes = 0
    for i, (pattern, replacement) in enumerate(patterns, start=1):
        text, changes = re.subn(pattern, replacement, text)
        print(f"Pattern {i}: '{pattern}' made {changes} changes.")
        total_changes += changes
    print(f"Total changes made: {total_changes}")
    return text

# Function to extract and format matches
def extract_and_format(text, match_pattern):
    """Extract matches using a regex pattern and format them."""
    matches = re.findall(match_pattern, text)
    return "\n".join([" ".join(match) for match in matches])

# Define patterns
patterns1 = [(r"\n(\))(.*)", r" \1 \2")]

match_pattern = r"([A-Z]{2}\d{15}) (.*) (.*) (ACC \(HQ\)|ACC|RPFC\-II|RPFC\-I|APFC) (\d{2}\/\d{2}\/\d{4}) (.*) (.*) (\d{2}\/\d{2}\/\d{4}) (\d+\.\d\d(?!\d)) (\d+\.\d\d(?!\d))"

patterns2 = [
    (r"(\d) (ACC \(HQ\)|ACC\-II|ACC\-I|ACC|RPFC\-II|RPFC\-I|APFC|REGULAR) ", r"\1----\2xxxx"),
    (r"(\d)\-\-\-\-(REGULAR)", r"\1----yyyy\2"),
    (r"REGULARxxxx", r"REGULARzzzz"),
    (r"xxxxREGULAR ", r"\tREGULAR\t"),
    (r"yyyyREGULARzzzz", r"\tREGULAR\t"),
    (r"xxxx", r"\t\t"),
    (r"----", r"\t"),
    (r" (\d{2}\/\d{2}\/\d{4}) (\d{2}\/\d{2}\/\d{4}) (\d{1,2}\.\d{2}) (\d{1,2}\.\d{2})", r"\t\1\t\2\t\3\t\4"),
    (r" (\d{2}\/\d{2}\/\d{4}) (\d{1,2}\.\d{2}) (\d{1,2}\.\d{2})", r"\t\1\t\t\2\t\3"),
    (r"([A-Z]{2}\d{15}) (.*) (\d{2}\/\d{2}\/\d{4})\t", r"\1\t\2\t\3\t"),
    (r"([A-Z]{2}\d{15})\t(.*) (ACC \(HQ\)|ACC|RPFC\-II|RPFC\-I|APFC)\t(\d{2}\/\d{2}\/\d{4})\t", r"\1\t\2\t\3\t\4\t"),
]

# Process all pages
all_formatted_text = ""
for page in reader.pages:
    text = page.extract_text()
    formatted_text = apply_patterns(text, patterns1)
    formatted_text = extract_and_format(formatted_text, match_pattern)
    formatted_text = apply_patterns(formatted_text, patterns2)
    all_formatted_text += formatted_text + "\n"

# Write to output file
with open(OUTPUT_PATH, "w") as text_file:
    text_file.write(all_formatted_text)

# Convert to DataFrame
lines = all_formatted_text.strip().split("\n")
columns = [
    "eid", "name_office","desig", "dob", "col2", "col3", "col4", "date1", "date2", "period1", "period2"
]  # Adjust column names as needed
data = [line.split("\t") for line in lines]
df = pd.DataFrame(data, columns=columns)

# Define patterns to remove
patterns = [
    "TRANSFERRED",
    "REGULAR PROMOTION AT SAME OFFICE",
    "REGULAR PROMOTION WITH TRANSFER",
    "AD-HOC PROMOTION WITH TRANSFER",
    "REPATRIATION FROM DEPUTATION AT SAME OFFICE",
    "BACK FROM FOREIGN SERVICE",
    "PROMOTION AT SAME OFFICE",
    "PROMOTION WITH TRANSFER",
]

# Function to remove patterns
def remove_pattern(text):
    if pd.isna(text):
        return None, text  # Handle NaN values gracefully
    for pattern in patterns:
        if isinstance(text, str) and text.startswith(pattern):
            new_col_value = pattern
            col4_value = text[len(pattern):].strip()
            return new_col_value, col4_value
    return None, text  # No pattern matched, return as is

df[['new_col4', 'col4']] = df['col4'].apply(lambda x: pd.Series(remove_pattern(x) if isinstance(x, str) else (None, x)))
df.to_csv(CSV_OUTPUT_PATH, index=False)
df


Pattern 1: '\n(\))(.*)' made 2 changes.
Total changes made: 2
Pattern 1: '(\d) (ACC \(HQ\)|ACC\-II|ACC\-I|ACC|RPFC\-II|RPFC\-I|APFC|REGULAR) ' made 90 changes.
Pattern 2: '(\d)\-\-\-\-(REGULAR)' made 9 changes.
Pattern 3: 'REGULARxxxx' made 9 changes.
Pattern 4: 'xxxxREGULAR ' made 27 changes.
Pattern 5: 'yyyyREGULARzzzz' made 9 changes.
Pattern 6: 'xxxx' made 54 changes.
Pattern 7: '----' made 90 changes.
Pattern 8: ' (\d{2}\/\d{2}\/\d{4}) (\d{2}\/\d{2}\/\d{4}) (\d{1,2}\.\d{2}) (\d{1,2}\.\d{2})' made 63 changes.
Pattern 9: ' (\d{2}\/\d{2}\/\d{4}) (\d{1,2}\.\d{2}) (\d{1,2}\.\d{2})' made 27 changes.
Pattern 10: '([A-Z]{2}\d{15}) (.*) (\d{2}\/\d{2}\/\d{4})\t' made 90 changes.
Pattern 11: '([A-Z]{2}\d{15})\t(.*) (ACC \(HQ\)|ACC|RPFC\-II|RPFC\-I|APFC)\t(\d{2}\/\d{2}\/\d{4})\t' made 90 changes.
Total changes made: 558
Pattern 1: '\n(\))(.*)' made 0 changes.
Total changes made: 0
Pattern 1: '(\d) (ACC \(HQ\)|ACC\-II|ACC\-I|ACC|RPFC\-II|RPFC\-I|APFC|REGULAR) ' made 27 changes.
Pattern 2: '(\d

,eid,name_office,desig,dob,col2,col3,col4,date1,date2,period1,period2,new_col4
0,AG181065124220894,ANITA SINHA DIXIT BENGALURU (CENTRAL),ACC (HQ),18/10/1965,ACC (HQ),,FARIDABAD ACC~BG BANGALORE,06/06/2023,05/06/2023,0.82,0.82,TRANSFERRED
1,AG181065124220894,ANITA SINHA DIXIT BENGALURU (CENTRAL),ACC (HQ),18/10/1965,,REGULAR,FARIDABAD ACC~HR FARIDABAD,20/03/2023,,0.21,0.82,PROMOTION AT SAME OFFICE
2,AG181065124220894,ANITA SINHA DIXIT BENGALURU (CENTRAL),ACC (HQ),18/10/1965,ACC,,HEAD OFFICE ACC~HR FARIDABAD,29/11/2018,28/11/2018,4.30,0.82,TRANSFERRED
3,AG181065124220894,ANITA SINHA DIXIT BENGALURU (CENTRAL),ACC (HQ),18/10/1965,ACC,REGULAR,HEAD OFFICE HEAD OFFICE DELHI,06/04/2018,,0.65,0.82,PROMOTION AT SAME OFFICE
4,AG181065124220894,ANITA SINHA DIXIT BENGALURU (CENTRAL),ACC (HQ),18/10/1965,ACC-II,REGULAR,HEAD OFFICE HEAD OFFICE DELHI,10/03/2016,,2.07,0.82,PROMOTION AT SAME OFFICE
...,...,...,...,...,...,...,...,...,...,...,...,...
112,SR230864027171290,SANDEEP KUMAR AGGARWAL NATRSS,ACC (HQ),23/08/1964,RPFC-II,,CHANDIGARH LUDHIANA LUDHIANA,17/05/1999,18/05/1999,3.70,0.82,TRANSFERRED
113,SR230864027171290,SANDEEP KUMAR AGGARWAL NATRSS,ACC (HQ),23/08/1964,RPFC-II,,BAREILLY CHANDIGARH CHANDIGARH,18/10/1996,17/10/1996,2.58,0.82,AD-HOC PROMOTION WITH TRANSFER
114,SR230864027171290,SANDEEP KUMAR AGGARWAL NATRSS,ACC (HQ),23/08/1964,APFC,,KANPUR BAREILLY BAREILLY,03/09/1993,30/08/1993,3.12,0.82,TRANSFERRED
115,SR230864027171290,SANDEEP KUMAR AGGARWAL NATRSS,ACC (HQ),23/08/1964,APFC,,CHANDIGARH KANPUR KANPUR,17/12/1990,11/01/1991,2.71,0.82,TRANSFERRED


# For Multiple PDF

In [136]:
from pypdf import PdfReader
import re
import pandas as pd
import glob

# Constants for file paths
INPUT_DIR = "<pdf dir path>"
OUTPUT_PATH = "<extracted csv path with file name>"

# Define patterns
patterns1 = [(r"\n(\))(.*)", r" \1 \2")]

match_pattern = r"([A-Z]{2}\d{15}) (.*) (ACC \(HQ\)|ACC|RPFC\-II|RPFC\-I|APFC) (\d{2}\/\d{2}\/\d{4}) (.*) (.*) (\d{2}\/\d{2}\/\d{4}) (\d+\.\d\d(?!\d)) (\d+\.\d\d(?!\d))"

patterns2 = [
    (r"(\d) (ACC \(HQ\)|ACC\-II|ACC\-I|ACC|RPFC\-II|RPFC\-I|APFC|REGULAR|TRANSFERRED|DEPUTATION|REVOCATION|SUSPENSION|BACK FROM|ON FOREIGN|REPATRIATION) ", r"\1----\2xxxx"),
    (r"(\d)\-\-\-\-(REGULAR)", r"\1----yyyy\2"),
    (r"REGULARxxxx", r"REGULARzzzz"),
    (r"xxxxREGULAR ", r"\tREGULAR\t"),
    (r"yyyyREGULARzzzz", r"\tREGULAR\t"),
    (r"\t(TRANSFERRED|SUSPENSION|DEPUTATION|REVOCATION|BACK FROM|ON FOREIGN|REPATRIATION)\t\t",r"\t\t\t\1 "),
    (r"xxxx", r"\t\t"),
    (r"----", r"\t"),
    (r" (\d{2}\/\d{2}\/\d{4}) (\d{2}\/\d{2}\/\d{4}) (\d{1,2}\.\d{2}) (\d{1,2}\.\d{2})", r"\t\1\t\2\t\3\t\4"),
    (r" (\d{2}\/\d{2}\/\d{4}) (\d{1,2}\.\d{2}) (\d{1,2}\.\d{2})", r"\t\1\t\t\2\t\3"),
    (r"([A-Z]{2}\d{15}) (.*) (\d{2}\/\d{2}\/\d{4})\t", r"\1\t\2\t\3\t"),
    (r"([A-Z]{2}\d{15})\t(.*) (ACC \(HQ\)|ACC|RPFC\-II|RPFC\-I|APFC)\t(\d{2}\/\d{2}\/\d{4})\t", r"\1\t\2\t\3\t\4\t"),
]

patterns = [
    "TRANSFERRED",
    "REGULAR PROMOTION AT SAME OFFICE",
    "REGULAR PROMOTION WITH TRANSFER",
    "AD-HOC PROMOTION WITH TRANSFER",
    "REPATRIATION FROM DEPUTATION AT SAME OFFICE",
    "BACK FROM FOREIGN SERVICE",
    "PROMOTION AT SAME OFFICE",
    "PROMOTION WITH TRANSFER",
    "BACK FROM STUDY LEAVE",
    "ON FOREIGN SERVICE",
    "REVOCATION OF SUSPENSION AT SAME OFFICE",
    "DEPUTATION AT SAME OFFICE",
    "SUSPENSION AT SAME OFFICE"

]

def apply_patterns(text, patterns):
    """Apply regex patterns and print the number of changes for each pattern."""
    total_changes = 0
    for i, (pattern, replacement) in enumerate(patterns, start=1):
        text, changes = re.subn(pattern, replacement, text)
        # print(f"Pattern {i}: '{pattern}' made {changes} changes.")
        total_changes += changes
    # print(f"Total changes made: {total_changes}")
    return text

def extract_and_format(text, match_pattern):
    """Extract matches using a regex pattern and format them."""
    matches = re.findall(match_pattern, text)
    return "\n".join([" ".join(match) for match in matches])

def process_pdf(pdf_path):
    """Process a single PDF file."""
    reader = PdfReader(pdf_path)
    all_formatted_text = ""
    for page in reader.pages:
        text = page.extract_text()
        formatted_text = apply_patterns(text, patterns1)
        formatted_text = extract_and_format(formatted_text, match_pattern)
        # if "AA070571115191199" in formatted_text:
        #     with open(OUTPUT_PATH0, "w") as text_file:
        #         text_file.write(formatted_text)
        formatted_text = apply_patterns(formatted_text, patterns2)
        all_formatted_text += formatted_text + "\n"

    # Convert to DataFrame
    lines = all_formatted_text.strip().split("\n")
    columns = [
        "eid", "name_office","desig", "dob", "col2", "col3", "col4", "date1", "date2", "period1", "period2"
    ]
    data = [line.split("\t") for line in lines]
    df = pd.DataFrame(data, columns=columns)

    # Remove patterns from 'col4'
    df[['new_col4', 'col4']] = df['col4'].apply(lambda x: pd.Series(remove_pattern(x) if isinstance(x, str) else (None, x)))

    return df

# Get list of PDF files
pdf_files = glob.glob(f"{INPUT_DIR}*.pdf")

# Process each PDF file and concatenate DataFrames
df_combined = pd.DataFrame()
for pdf_file in pdf_files:
    df_temp = process_pdf(pdf_file)
    df_combined = pd.concat([df_combined, df_temp], ignore_index=True)

# Save the combined DataFrame to CSV
df_combined.to_csv(OUTPUT_PATH, index=False)

# Save the combined DataFrame to JSON
JSON_OUTPUT_PATH = "extracted_data_combined.json"
df_combined.to_json(JSON_OUTPUT_PATH, orient="records", lines=True)

print(f"Processing complete. Combined data saved to {OUTPUT_PATH} and {JSON_OUTPUT_PATH}")

Processing complete. Combined data saved to /content/drive/MyDrive/EPFO/AGT/AGT data 2023-12/extracted_data_combined.csv


# Convert to JSON

In [161]:
def convert_to_json(df):
    """Convert DataFrame to desired JSON format."""
    json_data = []
    for _, row in df.iterrows():
        json_data.append({
            "eid": row['eid'],
            "name_office": row['name_office'],
            "desig": row['desig'],
            "dob": row['dob'],
            "col2": row['col2'],
            "col3": row['col3'],
            "col4": row['col4'],
            "date1": row['date1'],
            "date2": row['date2'],
            "period1": row['period1'],
            "period2": row['period2'],
            "new_col4": row['new_col4']
        })
    return json_data

import pandas as pd
import random
import hashlib
from datetime import datetime

# Function to generate a random ID (for anonymizing 'eid')
def generate_random_id(eid):
    return hashlib.sha256(str(eid).encode('utf-8')).hexdigest()[:16]

# Function to anonymize the 'dob' (masking exact day and month)
def anonymize_dob(dob):
    try:
        # Generate a random month and day but keep the year
        date_obj = datetime.strptime(dob, "%d/%m/%Y")
        random_month = random.randint(1, 12)
        random_day = random.randint(1, 28)  # Keeping it simple, assuming up to 28 days for any month
        return date_obj.replace(month=random_month, day=random_day).strftime("%d/%m/%Y")
    except Exception as e:
        return dob  # In case the format is wrong or it's empty, return as is

# Function to anonymize the 'name_office' (replacing with random office names)
def anonymize_name_office(office_name):
    splitted = office_name.split()
    office_names = "Office"
    return " ".join([splitted[0][0],splitted[1][0],office_names])

# Function to anonymize the 'desig' (replacing with generic designations)
def anonymize_desig(desig):
    designation_mapping = {
        "APFC": "I",
        "RPFC-II": "II",
        "RPFC-I": "III",
        "ACC": "IV",
        "ACC (HQ)": "V"
    }

    # Return the mapped value or default to "Unknown" if the designation is not found in the mapping
    return designation_mapping.get(desig, "Unknown")


# Function to anonymize the entire DataFrame
def anonymize_dataframe(df):
    df['eid'] = df['eid'].apply(generate_random_id)
    df['name_office'] = df['name_office'].apply(anonymize_name_office)
    df['desig'] = df['desig'].apply(anonymize_desig)
    df['dob'] = df['dob'].apply(anonymize_dob)
    df['date1'] = df['date1'].apply(anonymize_dob)
    df['date2'] = df['date2'].apply(anonymize_dob)
    return df

# Example: Anonymize the existing DataFrame (df_combined)
df_anonymized = anonymize_dataframe(df_combined)

# Now save the anonymized data to a new CSV or JSON
anonymized_csv_path = "extracted_data_anonymized.csv"
df_anonymized.to_csv(anonymized_csv_path, index=False)

anonymized_json_path = "extracted_data_anonymized.json"
json_data = convert_to_json(df_combined)
with open(anonymized_json_path, 'w') as json_file:
    json.dump(json_data, json_file, indent=4)

print(f"Anonymization complete. Anonymized data saved to {anonymized_csv_path} and {anonymized_json_path}")


Processing complete. Combined data saved to /content/drive/MyDrive/EPFO/AGT/AGT data 2023-12/extracted_data_combined.csv and /content/drive/MyDrive/EPFO/AGT/AGT data 2023-12/extracted_data_combined.json


# Trying encrypted json and decryption in html

In [168]:
import pandas as pd
import json
import base64
import hashlib
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from os import urandom



# Function to generate key from password using PBKDF2
def generate_key_from_password(password):
    # Use PBKDF2 with SHA-256 to generate a secure key from the password
    salt = urandom(16)  # Random salt for each password
    kdf = PBKDF2HMAC(algorithm=hashes.SHA256(), length=32, salt=salt, iterations=100000, backend=default_backend())
    key = kdf.derive(password.encode())  # Derive key from password
    return key, salt

# Function to encrypt data with AES-GCM
def encrypt_data(data, password):
    key, salt = generate_key_from_password(password)

    # Convert DataFrame to JSON string
    json_data = data.to_json(orient='records')
    json_bytes = json_data.encode('utf-8')  # Encode JSON data to bytes

    # Generate a random IV (Initialization Vector) for AES encryption
    iv = urandom(12)  # AES-GCM requires 12-byte IV

    # Set up the cipher for encryption
    cipher = Cipher(algorithms.AES(key), modes.GCM(iv), backend=default_backend())
    encryptor = cipher.encryptor()

    # Encrypt the data
    encrypted_data = encryptor.update(json_bytes) + encryptor.finalize()

    # Encode the encrypted data, IV, and tag in base64 for storage
    encrypted_base64 = base64.b64encode(iv + encryptor.tag + encrypted_data).decode('utf-8')

    # Return the encrypted data as base64 and the salt for later key derivation
    return encrypted_base64, salt

# Encrypt the DataFrame and save to JSON
password = '<your password>'  # This can be any password (remember this for decryption)
encrypted_data, salt = encrypt_data(df, password)

# Create a dictionary to store the encrypted data and salt
encrypted_json = {
    'encrypted_data': encrypted_data,
    'salt': base64.b64encode(salt).decode('utf-8')  # Encode salt to base64 for storage
}

# Save the encrypted data to a JSON file
with open('encrypted_data.json', 'w') as f:
    json.dump(encrypted_json, f)

# Print the password to be used in the HTML page for decryption
print(f"Password for decryption (use this in HTML): {password}")


Password for decryption (use this in HTML): <your password>
